# Lab14: Case Study
Austyn Smock
ID: 1050128
## Case Study: Blackjack
### Previous Program

In [2]:
#!/usr/bin/env python3

import random as r
from moneyfile import *

def card_deck(): 
    """
    This function creates a nested loop function that creates the deck of 52 cards
    """
    suit = ["Diamonds", "Clubs", "Hearts", "Spades"]    # Suit list
    rank = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"] # rank list
    card_deck = []
    for i in rank:
        for j in suit:
            if i == "Jack" or i == "Queen" or i == "King" or i == "10": 
                card_deck.append([(i + " of " + j), 10])
            elif i == "Ace":
                card_deck.append([(i + " of " + j), 1])
            else:
                card_deck.append([(i + " of " + j), int(i)])
    r.shuffle(card_deck)
    return card_deck


# Generate both player's hands, automate the dealer's, and automate ace behavior for dealer.
def deal(myDeck):
    """
    This function gets players and dealers hands, and creats ace behavior for the dealer
    """
    dealers_hand = []
    players_hand = []
    dealer_points = 0
    
    players_hand.append(myDeck.pop())
    players_hand.append(myDeck.pop())
    players_hand.append(myDeck.pop())
    dealers_hand.append(myDeck.pop())
    
    for card in dealers_hand:
        dealer_points += card[1]
    
    while dealer_points <= 17:
        dealers_hand.append(myDeck.pop())
        dealer_points = 0
        for card in dealers_hand:
            if card[1] != 1 or dealer_points >= 10:
                dealer_points += card[1]
            elif card[1] == 1 and dealer_points <= 10:
                dealer_points += 11
    return dealers_hand, players_hand


def welcome_message():
    """
    This function creates the game welcome message
    """
    print("BLACKJACK!")
    print("Blackjack payout is 3:2")
    print()
       
    
def more_chips():
    """
    This function asks user if they would like to buy more chips and checks the amount
    """
    money = read_money()
    if money < 5:
        chips = int(input("You're broke. Would you like to buy more chips?\n How many chips: "))
        
        # check input
        while chips > int(10000) or chips <= int(0):
            chips = int(input("Amount of chips must be greater than 0 and less than or equal to 10,000. Please try again.\n How many chips: "))
            print()
        new_money = money + chips
        write_money(str(new_money))
    else:
        new_money = money
    return new_money

def main():
    
    # displays the opening display
    welcome_message()
    
    try:
        money = read_money()
        print("Current money: ", money)
        more_chips()
    
    except FileNotFoundError:
        # Player's money
        money = int(input("Starting Money:\t"))
        
        # check if money is correct
        while money > 10000 or money <= 0:
            money = int(input("Starting money cannot be greater than 10,000 or less than or equal to 0. Try again: "))
            print()
        write_money(str(money))
      
    choice = "y"    
    while choice.lower() == "y":
        #Create and shuffle deck
        myDeck = card_deck()
        #Create both player's hands of cards
        dealers_hand, players_hand = deal(myDeck)
        #Initialize player's points and automate Ace behavior
        players_points = 0
        for card in players_hand:
            if card[1] != 1 or card[1] >= 10:
                players_points += card[1]
            elif card[1] == 1 and players_points <= 10:
                players_points += 11
        dealers_points = 0
        for card in dealers_hand:
            dealers_points += card[1]     
        
        # Betting
        while True:
            try:                                             # try clause
                bet_amount = float(input("Bet Amount: "))    # prompt user for bet amount
            except ValueError:                               # except ValueError clause
                print("Invalid entry. Please try again.")    # prints if invalid entry is inputted 
                continue
            while(bet_amount < 5 or bet_amount > 1000):   # checks if user input for bet amount is more than 5 and less than 1000
                bet_amount = float(input("Bet amount has to be more than 5 and less than 1000: "))
            while(bet_amount > money):                    # checks if user input for bet amount is less than players total money
                bet_amount = float(input("Bet amount cannot exceed current money. Please try again: "))
            break
        # Automate dealer bust
        if dealers_points > 21:
            print("Dealer bust! You win!")
            print()
            # Winning screen 
            print("DEALER'S CARDS:")
            for card in dealers_hand:
                print(card[0])
            print()
            print("YOUR CARDS:")
            for card in players_hand:
                print(card[0])

            money += bet_amount
            write_money(str(money))
            print(money)

            choice = input("Play again? (y/n): ")
            if choice.lower == "n":
                break
            else:
                continue

        # Game 
        print("DEALER'S SHOW CARD:")
        print(dealers_hand[0][0])

        print("YOUR CARDS:")
        for card in players_hand:
            print(card[0])
        
        # Hit or Stand
        hitOrStand = input("Hit or stand? (hit/stand): ")
        # Hit allows the player to add a card to their hand
        while hitOrStand.lower() == "hit":
            players_hand.append(myDeck.pop())
            # print cards
            for card in players_hand:
                print(card[0])
            # calculate points
            players_points = 0
            for card in players_hand:
                if card[1] != 1 or card[1] >= 10:
                    players_points += card[1]
                elif card[1] == 1 and players_points <= 10:
                    players_points += 11
            # Check if player busts with new card. 
            if players_points > 21: 
                print("You busted! Your bet is forfeit :(")
                money -= bet_amount
                write_money(str(money))
                more_chips()
                print(money)
                #Check if player wants to play again if they bust (lose)
                choice = input("Play again? (y/n): ")
                if choice.lower == "n":
                    break
                else:
                    hitOrStand = "out"
            else:
                hitOrStand = input("Hit or stand? (hit/stand): ")
            
        if hitOrStand.lower() == "stand":
            print("DEALER'S CARDS:")
            for card in dealers_hand:
                print(card[0])
            print()

            #End Game Screen
            print("YOUR CARDS")
            for card in players_hand:
                print(card[0])
            print("YOUR POINTS:\t", players_points)                                    
            print("DEALER'S POINTS:", dealers_points)
            
            if players_points == dealers_points:
                print("You pushed!")
                money = money
                write_money(str(money))
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points > dealers_points and players_points < 21:
                print("You win!")
                money += bet_amount
                write_money(str(money))
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points < dealers_points and players_points < 21:
                print("You lost!")
                money -= bet_amount
                
                write_money(str(money))
                
                money = more_chips()
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points == 21:
                print("You got a Blackjack!!")
                money += bet_amount * 1.5
                write_money(str(money))
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue  
    print("Bye!")
        
if __name__ == "__main__":
    main()

BLACKJACK!
Blackjack payout is 3:2

Starting Money:	2000
Bet Amount: 50
DEALER'S SHOW CARD:
5 of Hearts
YOUR CARDS:
Ace of Diamonds
2 of Clubs
Ace of Spades
Hit or stand? (hit/stand): stand
DEALER'S CARDS:
5 of Hearts
4 of Hearts
7 of Diamonds
3 of Clubs

YOUR CARDS
Ace of Diamonds
2 of Clubs
Ace of Spades
YOUR POINTS:	 13
DEALER'S POINTS: 19
You lost!
Money:  1950.0
Play again? (y/n)n
Bye!


### Hierarchy Chart for Moneyfile module
    main()
    - Def write_money function
        - writes player's money to money.txt file
    - Def read_money function
        - reads the players money from money.txt file
### Hierarchy Chart for blackjack
    main()
    - Def card_deck function
        - creates a nested loop function that creates the deck of 52 cards
    - Def deal(myDeck) function
        - This function gets players and dealers hands, and creats ace behavior for the dealer
    - Def welcome_message() function
        - displays welcome message
    - Def more_chips() function
        - asks user if they would like to buy more chips and checks the amount
    - Def main() function
### Expectations
-  Convert the program from procedural to object-oriented. This shouldn’t change the functionality of the code, but it should make the code more modular, reusable, and easier to maintain.

      - Use a Card class that provides attributes for storing the rank, suit, and points for a card. This class should also provide a method that returns a string that includes therank and suit of the card.

      - Use a Deck class that provides for a standard 52-card playing deck. This class should include methods that allow you to shuffle the deck and to deal cards from it.

      - Use a Hand class to store the dealer’s hand and the player’s hand. This class should include methods that allow you to add a card, get a card, count the number of cards,and get the total points for a hand of Blackjack.
      
### Creating my Card Class
#### UML
    Class Card
            Attributes:
                - suit
                - rank
                - points
            Methods:
                - init
                - showCard

In [5]:
class Card:
    """
    This class stores the suit, rank and points for my cards
    """
    def __init__(self, suit, rank, points):
        """
        Constructor for the Card class
        """
        self.suit = suit
        self.rank = rank
        self.points = points 
        
    def showCard(self):
        """
        Method prints the card's attributes
        """
        print(str(self.rank) + " of " + str(self.suit))
        
# testing 
testCard = Card("Spades", 8, 8)
testCard.showCard()

8 of Spades


### Creating my Deck Class
#### UML
    Class Card
            Attributes:
                - suit
                - rank
                - points
            Methods:
                - init
                - showCard
    Class Deck
            Attributes:
                - None
            Methods:
                - init
                - shuffleDeck
                - dealCard

In [9]:
import random as r

class Card:
    """
    This class stores the suit, rank and points for my cards
    """
    def __init__(self, suit, rank, points):
        """
        Constructor for the Card class
        """
        self.suit = suit
        self.rank = rank
        self.points = points 
        
    def showCard(self):
        """
        Method prints the card's attributes
        """
        print(str(self.rank) + " of " + str(self.suit))
        
class Deck: 
    """
    This class provides a 52-card playing deck
    """
    def __init__(self):
        self.deck = []
        suit = ["Diamonds", "Clubs", "Hearts", "Spades"]    # Suit list
        rank = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"] # rank list
        for i in rank:
            for j in suit:
                if i == "Jack" or i == "Queen" or i == "King" or i == "10": 
                    self.deck.append(Card(i, j, 10))
                elif i == "Ace":
                    self.deck.append(Card(i, j, 1))
                else:
                    self.deck.append(Card(i, j, int(i)))
                    
    def shuffleDeck(self):
        """
        Method shuffles deck
        """
        r.shuffle(self.deck)
    
    def dealCard(self):
        """
        Method deals card
        """
        return self.deck.pop()

# Testing
new_deck = Deck()
new_deck.shuffleDeck()
testCard = new_deck.dealCard()
testCard2 = new_deck.dealCard()
testCard.showCard()
testCard2.showCard()
print()
new_deck.shuffleDeck()
testCard = new_deck.dealCard()
testCard2 = new_deck.dealCard()
testCard.showCard()
testCard2.showCard()

Spades of King
Spades of 6

Diamonds of 2
Spades of 2


### Creating my Hand Class
#### UML
    Class Card
            Attributes:
                - suit
                - rank
                - points
            Methods:
                - init
                - showCard
    Class Deck
            Attributes:
                - None
            Methods:
                - init
                - shuffleDeck
                - dealCard
    Class Hand
            Attributes:
                - None
            Methods:
                - init
                - addCard
                - displayCard
                - showHand
                - calculatePoints

In [38]:
import random as r

class Card:
    """
    This class stores the suit, rank and points for my cards
    """
    def __init__(self, rank, suit, points):
        """
        Constructor for the Card class
        """
        
        self.rank = rank
        self.suit = suit
        self.points = points 
        
    def showCard(self):
        """
        Method prints the card's attributes
        """
        print(str(self.rank) + " of " + str(self.suit))
        
class Deck: 
    """
    This class provides a 52-card playing deck
    """
    def __init__(self):
        self.deck = []
        suit = ["Diamonds", "Clubs", "Hearts", "Spades"]    # Suit list
        rank = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"] # rank list
        for i in rank:
            for j in suit:
                if i == "Jack" or i == "Queen" or i == "King" or i == "10": 
                    self.deck.append(Card(i, j, 10))
                elif i == "Ace":
                    self.deck.append(Card(i, j, 1))
                else:
                    self.deck.append(Card(i, j, int(i)))
                    
    def shuffleDeck(self):
        """
        Method shuffles deck
        """
        r.shuffle(self.deck)
    
    def dealCard(self):
        """
        Method deals card
        """
        return self.deck.pop()
    
class Hand:
    """
    This class stores the dealer’s hand and the player’s hand
    """
    def __init__(self):
        """
        Method creates a list to store dealer's and player's hand
        """
        self.hand = []
        
    def addCard(self, deck):
        """
        Method adds a new card
        """
        self.hand.append(deck.dealCard())
        
    def displayCard(self):
        """
        Method displays card
        """
        self.hand[0].showCard()
    
    def showHand(self):
        """
        Method shows hand
        """
        for card in self.hand:
            card.showCard()
            
    def calculatePoints(self):
        """
        Method calculates points
        """
        value = 0
        aces = []
        for card in self.hand:
            if card.points == "Ace":
                aces.append(card)
                value += 11
            else:
                value += card.points
        if len(aces) == 0:
            return value
        while len(aces) > 0:
            if value <= 21:
                return value
            else:
                value -= 10
                aces.pop()
        return value
    
# Testing
new_deck = Deck()
new_deck.shuffleDeck()
player = Hand()
player.addCard(new_deck)
player.addCard(new_deck)
player.showHand()
value = player.calculatePoints()
print(value)
print()
new_deck.shuffleDeck()
player = Hand()
player.addCard(new_deck)
player.addCard(new_deck)
player.addCard(new_deck)
player.showHand()
print()
player.displayCard()
value = player.calculatePoints()
print(value) 

10 of Clubs
Ace of Diamonds
11

Jack of Clubs
4 of Spades
10 of Spades

Jack of Clubs
24


### Hierarchy Chart for Moneyfile module
    main()
    - Def write_money function
        - writes player's money to money.txt file
    - Def read_money function
        - reads the players money from money.txt file
### New Hierarchy Chart for blackjack
    main()
    - Def deal() function
        - This function gets players and dealers hands, and creats ace behavior for the dealer
        - Deck
            - shuffleDeck()
                -
            - dealCard()
            - Hand
                - addCard()
                - displayCard()
                - showHand()
                - calculatePoints()
            - Card
                - showCard()
    - Def welcome_message() function
        - displays welcome message
    - Def more_chips() function
        - asks user if they would like to buy more chips and checks the amount
    - Def main() function
### Final Code
I will putting it all together and adjusting previous code so that it works 

In [43]:
#!/usr/bin/env python3

import random as r
from moneyfile import *

class Card:
    """
    This class stores the suit, rank and points for my cards
    """
    def __init__(self, rank, suit, points):
        """
        Constructor for the Card class
        """
        self.rank = rank
        self.suit = suit
        self.points = points 
        
    def showCard(self):
        """
        Method prints the card's attributes
        """
        print(str(self.rank) + " of " + str(self.suit))
        
class Deck: 
    """
    This class provides a 52-card playing deck
    """
    def __init__(self):
        self.deck = []
        suit = ["Diamonds", "Clubs", "Hearts", "Spades"]    # Suit list
        rank = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"] # rank list
        for i in rank:
            for j in suit:
                if i == "Jack" or i == "Queen" or i == "King" or i == "10": 
                    self.deck.append(Card(i, j, 10))
                elif i == "Ace":
                    self.deck.append(Card(i, j, 1))
                else:
                    self.deck.append(Card(i, j, int(i)))
                    
    def shuffleDeck(self):
        """
        Method shuffles deck
        """
        r.shuffle(self.deck)
    
    def dealCard(self):
        """
        Method deals card
        """
        return self.deck.pop()
    
class Hand:
    """
    This class stores the dealer’s hand and the player’s hand
    """
    def __init__(self):
        """
        Method creates a list to store dealer's and player's hand
        """
        self.hand = []
        
    def addCard(self, deck):
        """
        Method adds a new card
        """
        self.hand.append(deck.dealCard())
        
    def displayCard(self):
        """
        Method displays card
        """
        self.hand[0].showCard()
    
    def showHand(self):
        """
        Method shows hand
        """
        for card in self.hand:
            card.showCard()
            
    def calculatePoints(self):
        """
        Method calculates points
        """
        value = 0
        aces = []
        for card in self.hand:
            if card.points == "Ace":
                aces.append(card)
                value += 11
            else:
                value += card.points
        if len(aces) == 0:
            return value
        while len(aces) > 0:
            if value <= 21:
                return value
            else:
                value -= 10
                aces.pop()
        return value
    
def welcome_message():
    """
    This function creates the game welcome message
    """
    print("BLACKJACK!")
    print("Blackjack payout is 3:2")
    print()
    
def more_chips():
    """
    This function asks user if they would like to buy more chips and checks the amount
    """
    money = read_money()
    if money < 5:
        chips = int(input("You're broke. Would you like to buy more chips?\n How many chips: "))
        
        # check input
        while chips > int(10000) or chips <= int(0):
            chips = int(input("Amount of chips must be greater than 0 and less than or equal to 10,000. Please try again.\n How many chips: "))
            print()
        new_money = money + chips
        write_money(str(new_money))
    else:
        new_money = money
    return new_money

# Generate both player's hands, automate the dealer's, and automate ace behavior for dealer.
def deal():
    """
    This function gets players and dealers hands, and creats ace behavior for the dealer
    """
    new_deck = Deck()
    new_deck.shuffleDeck()
    dealers_hand = Hand()
    players_hand = Hand()
    
    players_hand.addCard(new_deck)
    players_hand.addCard(new_deck)
    players_hand.addCard(new_deck)
    dealers_hand.addCard(new_deck)
    
    dealers_points = dealers_hand.calculatePoints()
    while dealers_points <= 17:
        dealers_hand.addCard(new_deck)
        dealers_points = dealers_hand.calculatePoints()
    return dealers_hand, players_hand, new_deck

def main():
    
    # displays the opening display
    welcome_message()
    
    try:
        money = read_money()
        print("Current money: ", money)
        more_chips()
    
    except FileNotFoundError:
        # Player's money
        money = int(input("Starting Money:\t"))
        
        # check if money is correct
        while money > 10000 or money <= 0:
            money = int(input("Starting money cannot be greater than 10,000 or less than or equal to 0. Try again: "))
            print()
        write_money(str(money))
      
    choice = "y"    
    while choice.lower() == "y":
        # Create both player's hands of cards
        dealers_hand, players_hand, deck = deal()
        players_points = players_hand.calculatePoints()
        dealers_points = dealers_hand.calculatePoints()
        
        # Betting
        while True:
            try:                                             # try clause
                bet_amount = float(input("Bet Amount: "))    # prompt user for bet amount
            except ValueError:                               # except ValueError clause
                print("Invalid entry. Please try again.")    # prints if invalid entry is inputted 
                continue
            while(bet_amount < 5 or bet_amount > 1000):   # checks if user input for bet amount is more than 5 and less than 1000
                bet_amount = float(input("Bet amount has to be more than 5 and less than 1000: "))
            while(bet_amount > money):                    # checks if user input for bet amount is less than players total money
                bet_amount = float(input("Bet amount cannot exceed current money. Please try again: "))
            break
        # Automate dealer bust
        if dealers_points > 21:
            print("Dealer bust! You win!")
            print()
            # Winning screen 
            print("DEALER'S CARDS:")
            dealers_hand.showHand()
     
            print("\nYOUR CARDS:")
            players_hand.showHand()
            money += bet_amount
            write_money(str(money))
            print(money)

            choice = input("Play again? (y/n): ")
            if choice.lower == "n":
                break
            else:
                continue

        # Game 
        print("DEALER'S SHOW CARD:")
        dealers_hand.displayCard()

        print("\nYOUR CARDS:")
        players_hand.showHand()
        
        # Hit or Stand
        hitOrStand = input("Hit or stand? (hit/stand): ")
        # Hit allows the player to add a card to their hand
        while hitOrStand.lower() == "hit":
            players_hand.addCard(deck)
            players_hand.showHand()
            players_points = players_hand.calculatePoints()
            
            # Check if player busts with new card. 
            if players_points > 21: 
                print("You busted! :(")
                money -= bet_amount
                write_money(str(money))
                more_chips()
                print(money)
                #Check if player wants to play again if they bust (lose)
                choice = input("Play again? (y/n): ")
                if choice.lower == "n":
                    break
                else:
                    hitOrStand = "out"
            else:
                hitOrStand = input("Hit or stand? (hit/stand): ")
            
        if hitOrStand.lower() == "stand":
            print("DEALER'S CARDS:")
            dealers_hand.showHand()
            
            #End Game Screen
            print("\nYOUR CARDS")
            players_hand.showHand()
            print("YOUR POINTS:\t", players_points)                                    
            print("DEALER'S POINTS:", dealers_points)
            
            if players_points == dealers_points:
                print("You pushed!")
                money = money
                write_money(str(money))
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points > dealers_points and players_points < 21:
                print("You win!")
                money += bet_amount
                write_money(str(money))
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points < dealers_points and players_points < 21:
                print("You lost!")
                money -= bet_amount
                
                write_money(str(money))
                
                money = more_chips()
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points == 21:
                print("You got a Blackjack!!")
                money += bet_amount * 1.5
                write_money(str(money))
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue  
    print("Bye!")
        
if __name__ == "__main__":
    main()

BLACKJACK!
Blackjack payout is 3:2

Current money:  2900.0
Bet Amount: 50
DEALER'S SHOW CARD:
Jack of Clubs

YOUR CARDS:
8 of Clubs
8 of Hearts
7 of Hearts
Hit or stand? (hit/stand): stand
DEALER'S CARDS:
Jack of Clubs
King of Clubs

YOUR CARDS
8 of Clubs
8 of Hearts
7 of Hearts
YOUR POINTS:	 23
DEALER'S POINTS: 20
Bet Amount: 50
Dealer bust! You win!

DEALER'S CARDS:
4 of Hearts
King of Hearts
3 of Spades
8 of Hearts

YOUR CARDS:
Queen of Hearts
2 of Spades
9 of Spades
2950.0
Play again? (y/n): y
Bet Amount: 50
Dealer bust! You win!

DEALER'S CARDS:
4 of Diamonds
2 of Hearts
7 of Clubs
Jack of Diamonds

YOUR CARDS:
3 of Diamonds
Queen of Diamonds
8 of Diamonds
3000.0
Play again? (y/n): n
Bye!


In [ ]:
#!/usr/bin/env python3

import random as r
from moneyfile import *

class Card:
    """
    This class stores the suit, rank and points for my cards
    """
    def __init__(self, rank, suit, points):
        """
        Constructor for the Card class
        """
        self.rank = rank
        self.suit = suit
        self.points = points 
        
    def showCard(self):
        """
        Method prints the card's attributes
        """
        print(str(self.rank) + " of " + str(self.suit))
        
class Deck: 
    """
    This class provides a 52-card playing deck
    """
    def __init__(self):
        self.deck = []
        suit = ["Diamonds", "Clubs", "Hearts", "Spades"]    # Suit list
        rank = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"] # rank list
        for i in rank:
            for j in suit:
                if i == "Jack" or i == "Queen" or i == "King" or i == "10": 
                    self.deck.append(Card(i, j, 10))
                elif i == "Ace":
                    self.deck.append(Card(i, j, 1))
                else:
                    self.deck.append(Card(i, j, int(i)))
                    
    def shuffleDeck(self):
        """
        Method shuffles deck
        """
        r.shuffle(self.deck)
    
    def dealCard(self):
        """
        Method deals card
        """
        return self.deck.pop()
    
class Hand:
    """
    This class stores the dealer’s hand and the player’s hand
    """
    def __init__(self):
        """
        Method creates a list to store dealer's and player's hand
        """
        self.hand = []
        
    def addCard(self, deck):
        """
        Method adds a new card
        """
        self.hand.append(deck.dealCard())
        
    def displayCard(self):
        """
        Method displays card
        """
        self.hand[0].showCard()
    
    def showHand(self):
        """
        Method shows hand
        """
        for card in self.hand:
            card.showCard()
            
    def calculatePoints(self):
        """
        Method calculates points
        """
        value = 0
        aces = []
        for card in self.hand:
            if card.points == "Ace":
                aces.append(card)
                value += 11
            else:
                value += card.points
        if len(aces) == 0:
            return value
        while len(aces) > 0:
            if value <= 21:
                return value
            else:
                value -= 10
                aces.pop()
        return value
    
def welcome_message():
    """
    This function creates the game welcome message
    """
    print("BLACKJACK!")
    print("Blackjack payout is 3:2")
    print()
    
def more_chips():
    """
    This function asks user if they would like to buy more chips and checks the amount
    """
    money = read_money()
    if money < 5:
        chips = int(input("You're broke. Would you like to buy more chips?\n How many chips: "))
        
        # check input
        while chips > int(10000) or chips <= int(0):
            chips = int(input("Amount of chips must be greater than 0 and less than or equal to 10,000. Please try again.\n How many chips: "))
            print()
        new_money = money + chips
        write_money(str(new_money))
    else:
        new_money = money
    return new_money

# Generate both player's hands, automate the dealer's, and automate ace behavior for dealer.
def deal():
    """
    This function gets players and dealers hands, and creats ace behavior for the dealer
    """
    new_deck = Deck()
    new_deck.shuffleDeck()
    dealers_hand = Hand()
    players_hand = Hand()
    
    players_hand.addCard(new_deck)
    players_hand.addCard(new_deck)
    players_hand.addCard(new_deck)
    dealers_hand.addCard(new_deck)
    
    dealers_points = dealers_hand.calculatePoints()
    while dealers_points <= 17:
        dealers_hand.addCard(new_deck)
        dealers_points = dealers_hand.calculatePoints()
    return dealers_hand, players_hand, new_deck

def main():
    
    # displays the opening display
    welcome_message()
    
    try:
        money = read_money()
        print("Current money: ", money)
        more_chips()
    
    except FileNotFoundError:
        # Player's money
        money = int(input("Starting Money:\t"))
        
        # check if money is correct
        while money > 10000 or money <= 0:
            money = int(input("Starting money cannot be greater than 10,000 or less than or equal to 0. Try again: "))
            print()
        write_money(str(money))
      
    choice = "y"    
    while choice.lower() == "y":
        # Create both player's hands of cards
        dealers_hand, players_hand, deck = deal()
        players_points = players_hand.calculatePoints()
        dealers_points = dealers_hand.calculatePoints()
        
        # Betting
        while True:
            try:                                             # try clause
                bet_amount = float(input("Bet Amount: "))    # prompt user for bet amount
            except ValueError:                               # except ValueError clause
                print("Invalid entry. Please try again.")    # prints if invalid entry is inputted 
                continue
            while(bet_amount < 5 or bet_amount > 1000):   # checks if user input for bet amount is more than 5 and less than 1000
                bet_amount = float(input("Bet amount has to be more than 5 and less than 1000: "))
            while(bet_amount > money):                    # checks if user input for bet amount is less than players total money
                bet_amount = float(input("Bet amount cannot exceed current money. Please try again: "))
            break
        # Automate dealer bust
        if dealers_points > 21:
            print("Dealer bust! You win!")
            print()
            # Winning screen 
            print("DEALER'S CARDS:")
            dealers_hand.showHand()
     
            print("\nYOUR CARDS:")
            players_hand.showHand()
            money += bet_amount
            write_money(str(money))
            print(money)

            choice = input("Play again? (y/n): ")
            if choice.lower == "n":
                break
            else:
                continue

        # Game 
        print("DEALER'S SHOW CARD:")
        dealers_hand.displayCard()

        print("\nYOUR CARDS:")
        players_hand.showHand()
        
        # Hit or Stand
        hitOrStand = input("Hit or stand? (hit/stand): ")
        # Hit allows the player to add a card to their hand
        while hitOrStand.lower() == "hit":
            players_hand.addCard(deck)
            players_hand.showHand()
            players_points = players_hand.calculatePoints()
            
            # Check if player busts with new card. 
            if players_points > 21: 
                print("You busted! :(")
                money -= bet_amount
                write_money(str(money))
                more_chips()
                print(money)
                #Check if player wants to play again if they bust (lose)
                choice = input("Play again? (y/n): ")
                if choice.lower == "n":
                    break
                else:
                    hitOrStand = "out"
            else:
                hitOrStand = input("Hit or stand? (hit/stand): ")
            
        if hitOrStand.lower() == "stand":
            print("DEALER'S CARDS:")
            dealers_hand.showHand()
            
            #End Game Screen
            print("\nYOUR CARDS")
            players_hand.showHand()
            print("YOUR POINTS:\t", players_points)                                    
            print("DEALER'S POINTS:", dealers_points)
            
            if players_points == dealers_points:
                print("You pushed!")
                money = money
                write_money(str(money))
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points > dealers_points and players_points < 21:
                print("You win!")
                money += bet_amount
                write_money(str(money))
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points < dealers_points and players_points < 21:
                print("You lost!")
                money -= bet_amount
                
                write_money(str(money))
                
                money = more_chips()
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue
            if players_points == 21:
                print("You got a Blackjack!!")
                money += bet_amount * 1.5
                write_money(str(money))
                print("Money: ", money)
                choice = input("Play again? (y/n)")
                if choice.lower == "n":
                    break
                else:
                    continue  
    print("Bye!")
        
if __name__ == "__main__":
    main()

BLACKJACK!
Blackjack payout is 3:2

Current money:  3100.0
Bet Amount: 100
DEALER'S SHOW CARD:
3 of Diamonds

YOUR CARDS:
4 of Diamonds
Queen of Diamonds
9 of Diamonds
Hit or stand? (hit/stand): stand
DEALER'S CARDS:
3 of Diamonds
Ace of Spades
Queen of Clubs
6 of Clubs

YOUR CARDS
4 of Diamonds
Queen of Diamonds
9 of Diamonds
YOUR POINTS:	 23
DEALER'S POINTS: 20
Bet Amount: 100
Dealer bust! You win!

DEALER'S CARDS:
King of Diamonds
6 of Spades
Jack of Spades

YOUR CARDS:
3 of Diamonds
King of Hearts
5 of Clubs
3200.0
Play again? (y/n): y
Bet Amount: y
Invalid entry. Please try again.
Bet Amount: 100
DEALER'S SHOW CARD:
Ace of Hearts

YOUR CARDS:
6 of Spades
King of Spades
2 of Diamonds
Hit or stand? (hit/stand): stand
DEALER'S CARDS:
Ace of Hearts
10 of Diamonds
Ace of Diamonds
7 of Hearts

YOUR CARDS
6 of Spades
King of Spades
2 of Diamonds
YOUR POINTS:	 18
DEALER'S POINTS: 19
You lost!
Money:  3100.0
Play again? (y/n)y
Bet Amount: 100
Dealer bust! You win!

DEALER'S CARDS:
8 of Clu